In [1]:
import random
import numpy as np

In [2]:
def criar_caixa(caixas,nome_caixa, comprimento, largura, altura):
    caixas[nome_caixa] = [comprimento, largura, altura]
    return caixas

def criar_item(itens,nome_item, comprimento, largura, altura):
    itens[nome_item] = [comprimento, largura, altura, 0]
    return itens

def item_combination(item):
    if item[3] == 0:
        lista = [item[2], item[1], item[0], 1] 
        return lista, 1
    elif item[3] == 1:
        lista = [item[1], item[0], item[2], 2]
        return lista, 1
    elif item[3] == 2:
        lista = [item[0], item[2], item[1], 3]
        return lista, 1
    elif item[3] == 3:
        lista = [item[1], item[2], item[0], 4]
        return lista, 1
    elif item[3] == 4:
        lista = [item[2], item[0], item[1], 5]
        return lista, 1
    else:
        lista = [item[0], item[1], item[2], 0]
        return lista, 1

def LS(caixa, itens):
    escolha = 1 #random.randrange(4, 5)
    solucao = []
    if escolha == 1:
        itens_aux = list(itens.values())
        random.shuffle(itens_aux)
        sol, num_bins = corner_heuristic([], itens_aux, caixa)
        solucao.append(sol)
        solucao.append(num_bins)
        return solucao

In [3]:
def corner_heuristic(itens_empacotados, itens, caixa, index_caixa = 1):
    if type(itens) == type({'a':'b'}):
        itens_por_dim = list(itens.values())
    else:
        itens_por_dim = itens
    soma = 1
    save_itens_por_dim = itens_por_dim.copy()
    save = 0
    corner_point = [[0,0,0]]
    itens_empacotados.append(index_caixa)
    
    for item in itens_por_dim:
        item_save = item.copy()
        if max(item) > max(caixa) or min(item) > min(caixa):
            return [], 0
        for point in corner_point:
            i = 0
            while point[0] + item[0] > caixa[0] or point[1] + item[1] > caixa[1] or point[2] + item[2] > caixa[2]:
                if i > 6:
                    break
                item, save = item_combination(item)
                i = i + 1
            
            ponta1 = point[0] + item[0]
            ponta2 = point[1] + item[1]
            ponta3 = point[2] + item[2]
            
            if ponta1 <= caixa[0] and ponta2 <= caixa[1] and ponta3 <= caixa[2]:
                corner_point.append([point[0]+item[0], point[1], point[2]])
                corner_point.append([point[0], point[1]+item[1], point[2]])
                corner_point.append([point[0], point[1], point[2]+item[2]])
                if save == 1:
                    itens_empacotados.append(item)
                    save_itens_por_dim.remove(item_save)
                else:
                    itens_empacotados.append(item)
                    save_itens_por_dim.remove(item_save)
                corner_point.remove(point)
                break

        
    if len(save_itens_por_dim) > 0:
        if index_caixa > 500:
            return [], 0
        index_caixa = index_caixa + 1
        itens_empacotados, corner = corner_heuristic(itens_empacotados, save_itens_por_dim, caixa, index_caixa)
        soma = soma + corner
    return itens_empacotados, soma

In [4]:
def simulated_annealing(empacotamento, itens, caixa):
    T_min = 10**(-2) ## > 10^5
    T0 = 10000
    SA_max = 2000 ## [100, 2000]
    alpha = 0.999 ## [0.8, 0.9999]
    iter = 0
    T = T0
    melhor_sol = empacotamento
    corrente = empacotamento
    while T > T_min:
        while iter < SA_max:
            iter += 1
            corrente1 = LS(caixa, itens)
            delta = corrente1[1] - corrente[1]
            if delta <= 0:
                corrente = corrente1
                if corrente[1] <= melhor_sol[1]:
                    sol_otm = corrente
                else:
                    x = random.random()
                    if x < np.e**(-delta/T):
                        corrente = corrente1
        T = T*alpha
    return sol_otm

In [7]:
lista_leng = [20.3, 20.3]
lista_wid = [5.1, 5.1]
lista_heig = [15.2, 15.2]
itens = {}
caixas = {}

for i in range(len(lista_leng)):
    itens = criar_item(itens, 'Item' + str(i), lista_leng[i], lista_wid[i], lista_heig[i])

caixa = [21.6, 20.3, 10.4]
sol = LS(caixa, itens)
sol = simulated_annealing(sol, itens, caixa)
print(sol)

[[1, [20.3, 15.2, 5.1, 4], [20.3, 15.2, 5.1, 4]], 1]
